# Sample CloudSql Connection

# Need to set up for each instance - Add your VM's IP to the Authorized Networks
**1. Find the external IP of your JupyterLab VM**
 * Go to VM Instances
 * Find your JupyterLab VM
 * Copy the External IP address (looks like 34.91.100.45).

**2. Add the VM's IP to your Cloud SQL authorized networks**
* Go to Cloud SQL instances. https://console.cloud.google.com/compute/instances?project=adsp-34002-on02-sopho-scribe&authuser=1
* Click your instance.
* Click Connections in the left sidebar.
* Scroll to Authorized networks → Add network.
* Name: anything like jupyterlab-vm
* Network: paste the external IP you just copied (e.g., 34.91.100.45/32)
* IMPORTANT: Add /32 to allow only that single IP.
* Click Save.

It will take ~30 seconds to update.

In [13]:
#%pip install -q google-cloud-secret-manager==2.23.3
#%pip install -q SQLAlchemy==2.0.40

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install psycopg2-binary sqlalchemy pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import create_engine, text

#### Retrieve credentials from Secret Manager

In [3]:
def access_secret(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{PROJECT_ID}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(name=name)
    return response.payload.data.decode("UTF-8")

#### Configuration

In [5]:
PROJECT_ID = 'adsp-34002-on02-sopho-scribe'
REGION = 'us-central1'
#DB_NAME = 'crm'
DB_NAME = 'postgres'
DB_USER = access_secret('cloudSqlUser')
DB_PASSWORD = access_secret('cloudSqlUserPassword')
DB_HOST = '35.232.92.211'
DB_PORT = '5432'

In [4]:
from sqlalchemy import create_engine
import pandas as pd

db_user = "lilycampbell"
db_pass = "moneypenny"
db_name = "crm"
host = '35.232.92.211'  
port = 5432

engine = create_engine(f"postgresql://{db_user}:{db_pass}@{host}:{port}/{db_name}")

# Test connection
df = pd.read_sql("SELECT NOW()", engine)
print(df)

                               now
0 2025-04-24 06:10:36.628546+00:00


#### Create SQLAlchemy engine

In [6]:
engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}',
    connect_args={'sslmode': 'require'}
)

#### Generate & insert sample data

In [7]:
# Create sample data
sample_data = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'score': [88, 92, 85]
})

# Create table and insert data
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS test_scores (
            id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            score INT
        )
    """))

    for _, row in sample_data.iterrows():
        conn.execute(
            text("INSERT INTO test_scores (name, score) VALUES (:name, :score)"),
            {"name": row['name'], "score": row['score']}
        )

#### Read back the data

In [8]:
df_result = pd.read_sql("SELECT * FROM test_scores", con=engine)
print(df_result)

   id     name  score
0   1    Alice     88
1   2      Bob     92
2   3  Charlie     85


In [9]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")

Notebook last execution time: Thu, 24 April 2025 01:12:16
